# Cluster Analysis for Frizz Collab #
Aaron Au - Yip Lab 2024

This script is designed to import HDF5 files from Picasso filtered or HDF filtered images and process localization data with certain clustering data

In [557]:
import h5py
import locan as lc
from locan.data.filter import random_subset
import numpy as np
import os
import pandas as pd
import scipy.spatial as spatial
import shutil
import skimage.measure as measure
import skimage.morphology as morph
import tifffile as tf
from tqdm.notebook import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time

In [530]:
folder = "\\\\5.5.5.250\\share01\\yipgroup\\Au_Aaron\\02164-FRIZZ_SM\\" #Location where hdf files are stored
save_folder = "\\\\5.5.5.250\\share01\\yipgroup\\Au_Aaron\\02164-FRIZZ_SM\\processed\\" #Location to save processed data

times = ["290524", "210524", "040924", "030724"]
files = ["A","B","C","D","E","F","G", "H"]
ite = 5
suffix = ".hdf5"

px = 65. #effective pixel size (nm)
num_subsets = 1
n_points = 10000 # Limit detection per image

d_nn = 5 #Distance defined as same blink

#For NN analysis
ks = [1,2,4,8,16,32] #K values



In [161]:
#Process clustering data
cols = ["file", "time", "trial", "sub", "NN", "Voronoi", "HDBSCAN", "ripleyH"]

data = []
for time in times:
    for file in files:
        for i in range(1,ite+1):
            #Import localization data
            df = pd.DataFrame(np.array(h5py.File(folder  +  time + "\\" + file +"_"+ str(i)+suffix)['locs']))
            #Convert from camera pixel data to distances
            df['lpx']= df['lpx']*px
            df['lpy']= df['lpy']*px
            df['x'] = df['x']*px
            df['y'] = df['y']*px

            #Convert to locan data
            locdata = lc.LocData.from_dataframe(dataframe = df.rename(columns={"x": "position_x", "y": "position_y",
                                                                              "lpx": "uncertainty_x", "lpy": "uncertainty_y"}))
            #Remove reoccuring blinks
            diff = 10;
            while diff != 0:
                nn = lc.NearestNeighborDistances(k=1)
                nn.compute(locdata)
                nn.results = nn.results.rename(index = dict(zip(nn.results.index, locdata.dataframe.index)))
                start = len(locdata.dataframe)
                mask = (nn.results["nn_distance"] < d_nn) & (nn.results["nn_index"] < locdata.dataframe.index) #want to remove these ones
                locdata.dataframe = locdata.dataframe.loc[mask == False]
                diff = start - len(locdata.dataframe)       
            
            #Process for each image + subset
            subsets = []
            for j in range(num_subsets):
                subsets.append(lc.random_subset(locdata, n_points = n_points))
        
            j = 0
            for subset in subsets: 
                #NEAREST NEIGHBOUR ANALYSIS
                nns = []
                print(f'processing data {len(subset)}')
                for k in ks:
                    nn = lc.NearestNeighborDistances(k=k)
                    nn.compute(subset)
                    nns.append(nn)

                #VORONOI ANALYSIS
                vor = spatial.Voronoi(subset.coordinates)

                #HDBSCAN ANALYSIS
                noise, clust = lc.cluster_hdbscan(subset, min_samples=2)
    
                #RIPLEYS H ANALYSIS
                rhf = lc.RipleysHFunction(radii=np.linspace(0,500,51)).compute(locdata, other_locdata = subset)

                #Add to data
                data.append([file, time, i, j, nns, vor, (noise, clust), rhf])
                j+=1

                #Uncomment for progress
                #print(f'done {time, file, i,j}')

# Save processed data
df = pd.DataFrame(data, columns = cols)
df.to_pickle(folder + 'pickled_data.pkl')

processing data 10000
done ('290524', 'A', 1, 1)
processing data 10000
done ('290524', 'A', 2, 1)
processing data 10000
done ('290524', 'A', 3, 1)
processing data 10000
done ('290524', 'A', 4, 1)
processing data 10000
done ('290524', 'A', 5, 1)
processing data 10000
done ('290524', 'B', 1, 1)
processing data 10000
done ('290524', 'B', 2, 1)
processing data 10000
done ('290524', 'B', 3, 1)
processing data 10000
done ('290524', 'B', 4, 1)
processing data 10000
done ('290524', 'B', 5, 1)
processing data 10000
done ('290524', 'C', 1, 1)
processing data 10000
done ('290524', 'C', 2, 1)
processing data 10000
done ('290524', 'C', 3, 1)
processing data 10000
done ('290524', 'C', 4, 1)
processing data 10000
done ('290524', 'C', 5, 1)
processing data 10000
done ('290524', 'D', 1, 1)
processing data 10000
done ('290524', 'D', 2, 1)
processing data 10000
done ('290524', 'D', 3, 1)
processing data 10000
done ('290524', 'D', 4, 1)
processing data 10000
done ('290524', 'D', 5, 1)
processing data 1000

In [328]:
df = pd.read_pickle(folder + 'pickled_data.pkl')

In [464]:
#Combine Voronoi, NN data into a table
#into a single table
cols = ['file', 'time', 'trial', 'position_x', 'position_y', 'NN', 'Vor_area']
data = []
for file in files:
    for time in times:
        for trial in range(5):
            matches = df[(df['file'] == file) & (df['time'] == time) & (df['trial'] == trial+1)]
            #Extract Voronoi data
            vor = matches.iloc[0]['Voronoi']
            #Get volume of all closed polygons
            vol_Vor = np.zeros(vor.npoints)
            for idx, i_region in enumerate(vor.point_region):
                if -1 not in vor.regions[i_region]: #Remove open polygons
                    vol_Vor[idx] = spatial.ConvexHull(vor.vertices[vor.regions[i_region]]).volume
            #Extract NN data              
            #['file', 'time', 'trial', 'position_x', 'position_y', 'NN', 'Vor_area']
            nn = matches.iloc[0]['NN']
            for pt in range(vor.npoints):
                data.append([file, time, trial+1, vor.points[pt,0], vor.points[pt,1]
                             , [nn[i].results['nn_distance'][pt] for i in range(len(ks))]
                             , vol_Vor[pt]])

#Save data
df_combo = pd.DataFrame(data, columns = cols)
df_combo.to_pickle(folder + 'pickled_combo_data.pkl')

In [512]:
#EXPORT nn data from combo
bins = np.linspace(5,505,51)

for file in files:
    writer = pd.ExcelWriter(save_folder + file + "_nns.xlsx", engine='xlsxwriter')
    j = 0
    data = np.zeros([(len(times)*5)+1,len(bins)-1, len(ks)])
    cols = ['distance']
    for time in times:
        for t in range(5):
            matches = df_combo[(df_combo['file']==file) & (df_combo['time']==time) & (df_combo['trial']==t+1)]
            nns = np.stack(matches['NN'].to_numpy())
            cols.append(time + "_" +file + str(t+1))
            for i in range(len(ks)):
                data[j*5+t+1,:, i],_ = np.histogram(nns[:,i], bins = bins, weights=np.ones_like(nns[:,i]) / len(nns[:,i]), density = True)
        j+=1
    #Export Data
    for i in range(len(ks)):
        data[0,:,i] = bins[:-1]
        df_temp = pd.DataFrame(data=data[:,:,i].T, columns = cols)
        df_temp.to_excel(writer, sheet_name = str(ks[i]), index = False)
    writer.close()                              

In [528]:
#Export vor data from combo
bins = np.linspace(5,10000,501)

for file in files:
    writer = pd.ExcelWriter(save_folder + file + "_vor.xlsx", engine='xlsxwriter')
    j = 0
    data = np.zeros([(len(times)*5)+1,len(bins)-1])
    cols = ['distance']
    for time in times:
        for t in range(5):
            matches = df_combo[(df_combo['file']==file) & (df_combo['time']==time) & (df_combo['trial']==t+1)]
            vor_A = np.stack(matches['Vor_area'].to_numpy())
            cols.append(time + "_" +file + str(t+1))
            for i in range(len(ks)):
                data[j*5+t+1,:],_ = np.histogram(vor_A, bins = bins, weights=np.ones_like(vor_A) / len(vor_A), density = True)
        j+=1
    #Export Data
    data[0,:] = bins[:-1]
    df_temp = pd.DataFrame(data=data.T, columns = cols)
    df_temp.to_excel(writer, sheet_name = 'AREAS', index = False)
    writer.close()

In [575]:
#Compare NNs vs Vor data
cols = ['file', 'time', 'trial', 'position_x', 'position_y', 'Vor_area', 'NN1', 'NN2', 'NN4','NN8', 'NN16', 'NN32']
data = []

writer = pd.ExcelWriter("nnvsvor.xlsx", engine='xlsxwriter')
for i, row in df_combo.iterrows():
    if row['NN'][0] > 0.1:
        data.append([row['file'], row['time'], row['trial'], row['position_x'], row['position_y'], row['Vor_area'], 
                row['NN'][0], row['NN'][1], row['NN'][2], row['NN'][3], row['NN'][4], row['NN'][5]])
    

df_temp = pd.DataFrame(data=data, columns = cols)
if len(df_temp) > 1000000:
    df_temp[:1000000].to_excel(writer, sheet_name = 'pt1', index = False)
    df_temp[1000000-len(df_temp):].to_excel(writer, sheet_name = 'pt2', index = False)
else:
    df_temp.to_excel(writer, sheet_name = 'data', index = False)
writer.close()

ValueError: This sheet is too large! Your sheet size is: 1307397, 12 Max sheet size is: 1048576, 16384

In [525]:
#Combine three algorithms together
hist_range = (5,30000)
bins = 101

for file in files:
    for time in times:
        for trial in range(5):
            fig, ax = plt.subplots(figsize=(20,20))
            matches = df[(df['file'] == file) & (df['time'] == time) & (df['trial'] == trial+1)]
            #Extract Voronoi data
            vor = matches.iloc[0]['Voronoi']
            #Get volume of all closed polygons
            vol_Vor = np.zeros(vor.npoints)
            for idx, i_region in enumerate(vor.point_region):
                if -1 not in vor.regions[i_region]: #Remove open polygons
                    vol_Vor[idx] = spatial.ConvexHull(vor.vertices[vor.regions[i_region]]).volume
            
            #hists[i,:], _ = np.histogram(vol_Vor, range=hist_range, bins = bins, weights=np.ones_like(vol_Vor) / len(vol_Vor), density = True)
            
            #Normalize colour bar
            norm = mpl.colors.Normalize(vmin = min(vol_Vor), vmax=300000, clip=True)
            mapper = cm.ScalarMappable(norm=norm, cmap=cm.inferno)
            #Plot Voronoi map with colour
            spatial.voronoi_plot_2d(vor, ax = ax, show_points = False, show_vertices=False)
            for r in range(len(vor.point_region)):
                region = vor.regions[vor.point_region[r]]
                if not -1 in region:
                    polygon = [vor.vertices[i] for i in region]
                    ax.fill(*zip(*polygon), color=mapper.to_rgba(vol_Vor[r]))
            
            nn = matches.iloc[0]['NN'][0]
            norm2 = mpl.colors.Normalize(vmin = 0, vmax=250, clip=True)
            mapper2 = cm.ScalarMappable(norm=norm2, cmap=cm.viridis_r)
            ax.scatter(vor.points[:,0], vor.points[:,1], c=nn.results["nn_distance"], s=8, norm=norm2, cmap=cm.viridis_r)
                       
            #ax.get_legend().remove()
            fig.savefig(save_folder+time+"_"+file+"_"+str(trial)+".png", dpi=300)
            fig.clf()

C:\Users\YipGroup\AppData\Local\Temp\ipykernel_1776\3174165167.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(20,20))


<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

...

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>